In [2]:
pip install customtkinter

Note: you may need to restart the kernel to use updated packages.Collecting customtkinter

   ---------------------------------------- 0/2 [darkdetect]
   -------------------- ------------------- 1/2 [customtkinter]
   -------------------- ------------------- 1/2 [customtkinter]
   -------------------- ------------------- 1/2 [customtkinter]
   -------------------- ------------------- 1/2 [customtkinter]
   -------------------- ------------------- 1/2 [customtkinter]
   -------------------- ------------------- 1/2 [customtkinter]
   -------------------- ------------------- 1/2 [customtkinter]
   -------------------- ------------------- 1/2 [customtkinter]
   ---------------------------------------- 2/2 [customtkinter]




In [ ]:
import customtkinter as ctk
from PIL import Image
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# --- 1. SETUP & ASSET LOADING ---
# Ensure stopwords are available
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
port_stem = PorterStemmer()

try:
    # Load the trained model and the vectorizer
    model = pickle.load(open('trained_model.sav', 'rb'))
    vectorizer = pickle.load(open('vectorizer.sav', 'rb'))
    print("✅ Model and Vectorizer loaded successfully.")
except FileNotFoundError:
    print("❌ Error: 'trained_model.sav' or 'vectorizer.sav' not found in this folder.")

# --- 2. NLP PRE-PROCESSING FUNCTION ---
def stemming(content):
    # Standardizing the text to match the training data format
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower().split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stop_words]
    return ' '.join(stemmed_content)

# --- 3. THE INTERFACE CLASS ---
ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("blue")

class SentimentAI(ctk.CTk):
    def __init__(self):
        super().__init__()

        # Window Settings
        self.title("AI Sentiment Intelligence")
        self.geometry("900x650")
        self.resizable(False, False)

        # Background Image
        try:
            self.bg_image = ctk.CTkImage(Image.open("background.png"), size=(900, 650))
            self.bg_label = ctk.CTkLabel(self, image=self.bg_image, text="")
            self.bg_label.place(relx=0, rely=0, relwidth=1, relheight=1)
        except:
            self.configure(fg_color="#0d1117")

        # Smaller Glass Panel (0.6 width for a sleeker look)
        self.main_frame = ctk.CTkFrame(self, corner_radius=30, 
                                      fg_color=("#ffffff", "#10141b"), 
                                      border_width=1, border_color="#30363d")
        self.main_frame.place(relx=0.5, rely=0.5, anchor="center", relwidth=0.6, relheight=0.75)

        # Header
        self.title_label = ctk.CTkLabel(self.main_frame, text="Sentiment Analysis", 
                                       font=ctk.CTkFont(family="Inter", size=28, weight="bold"))
        self.title_label.pack(pady=(35, 5))
        
        self.status = ctk.CTkLabel(self.main_frame, text="● Neural Network Active", 
                                  text_color="#2ecc71", font=ctk.CTkFont(size=12))
        self.status.pack(pady=(0, 20))

        # Input Area
        self.input_text = ctk.CTkTextbox(self.main_frame, width=380, height=120, 
                                        corner_radius=15, border_width=1, 
                                        fg_color="#0d1117", border_color="#1f6aa5",
                                        font=("Inter", 14))
        self.input_text.pack(pady=10)
        self.input_text.insert("0.0", "Type a social media post here...")

        # Analyze Button
        self.btn = ctk.CTkButton(self.main_frame, text="ANALYZE EMOTION", 
                                command=self.process_sentiment, height=50, width=280,
                                corner_radius=15, fg_color="#1f6aa5", hover_color="#145dbf",
                                font=ctk.CTkFont(size=15, weight="bold"))
        self.btn.pack(pady=25)

        # Confidence Meter
        self.conf_text = ctk.CTkLabel(self.main_frame, text="Prediction Confidence", font=ctk.CTkFont(size=11))
        self.conf_text.pack()
        self.progress = ctk.CTkProgressBar(self.main_frame, width=320, height=10, corner_radius=5)
        self.progress.set(0)
        self.progress.pack(pady=(5, 25))

        # Final Result Label
        self.result_label = ctk.CTkLabel(self.main_frame, text="Result: Waiting", 
                                        font=ctk.CTkFont(size=24, weight="bold"),
                                        text_color="gray")
        self.result_label.pack()

        # Footer Clear Button
        self.clear_btn = ctk.CTkLabel(self.main_frame, text="Clear text", 
                                     text_color="gray", cursor="hand2", font=ctk.CTkFont(size=11, underline=True))
        self.clear_btn.pack(pady=20)
        self.clear_btn.bind("<Button-1>", lambda e: self.input_text.delete("0.0", "end"))

    def process_sentiment(self):
        # 1. Capture text
        user_text = self.input_text.get("0.0", "end").strip()
        if not user_text or user_text == "Type a social media post here...":
            return

        # 2. NLP Pipeline
        # Diagram: 
        clean_text = stemming(user_text)
        vectorized_data = vectorizer.transform([clean_text])

        # 3. Model Inference
        # We use predict_proba to get the confidence percentage
        probs = model.predict_proba(vectorized_data)[0] 
        prediction = model.predict(vectorized_data)[0]
        confidence = max(probs)

        # 4. UI Update
        self.progress.set(confidence)
        if prediction == 1:
            self.result_label.configure(text="POSITIVE SENTIMENT 😊", text_color="#2ecc71")
            self.progress.configure(progress_color="#2ecc71")
            self.input_text.configure(border_color="#2ecc71")
        else:
            self.result_label.configure(text="NEGATIVE SENTIMENT 😠", text_color="#e74c3c")
            self.progress.configure(progress_color="#e74c3c")
            self.input_text.configure(border_color="#e74c3c")

if __name__ == "__main__":
    app = SentimentAI()
    app.mainloop()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\net\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ Model and Vectorizer loaded successfully.
